# Set Up and Imports

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
# NOTE: To be able to access the shared files, you need to go to Drive and click
# "Add shortcut to Drive" on the options for the shared folder to be able to access it when mounted

Mounted at /content/gdrive


In [ ]:
# Paths to the "Project Code" folder
path_andrew = "/content/gdrive/MyDrive/CS 7643/Project/Project Code"
path_gillian = "/content/gdrive/MyDrive/CS 7643/Project Code"
path_blake = "/content/gdrive/MyDrive/CS 7643/Project Code"
path_isaac = "/content/gdrive/MyDrive/classes/CS 7643/Project Code"
path_bryan = "/content/gdrive/MyDrive/Georgia Tech/CS 7643/CS 7643/Project Code"

In [ ]:
# change this to your path when running
path = path_gillian
%cd {path}

/content/gdrive/.shortcut-targets-by-id/1lmYZ9dBUQsLKZApoHNFRjbtE_VA1kGF4/CS 7643/Project Code


In [ ]:
# pip installs
!pip install transformers
!pip install torcheval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.4/158.4 kB 589.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 11.8 MB/s eta 0:00:00


In [ ]:
%load_ext autoreload
%autoreload 2

# torch
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torcheval.metrics.functional import binary_auroc

# data
from HMDataset import HMDataset, HMDataset_H5

# model
from transformers import AutoTokenizer, BertForTokenClassification, BertTokenizer,  BertForSequenceClassification
import torch
from torch.optim import AdamW
from transformers import AutoImageProcessor, ResNetForImageClassification

# general
from PIL import Image
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# gpu check
print(torch.__version__)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device=='cuda')

2.0.0+cu118
True


# Data I hardly know her?

In [ ]:
# Hyperparameters
batch_size = 32
lr = 1e-5
epochs = 7

In [ ]:
transform_h5 = transforms.ToTensor() # everything in HDF5 is already pre-resized to 256 x 256

train_h5_dataset = HMDataset_H5(h5_file=path + "/training_db.h5", transform=transform_h5)
train_h5_dataloader = DataLoader(train_h5_dataset, batch_size=batch_size, shuffle=True)

validate_dataset = HMDataset_H5(h5_file=path + "/validation_db.h5", transform=transform_h5)
validate_dataloader = DataLoader(validate_dataset, batch_size=batch_size, shuffle=True)

# Utility Functions

In [ ]:
def get_metrics_and_losses(losses, predictions, labels):
  average_loss = losses.mean().item()
  accuracy = (predictions == labels).sum().item() / labels.numel()
  auroc = binary_auroc(predictions, labels)

  return average_loss, accuracy, auroc

In [ ]:
def weighted(inputs):
  output = torch.empty(inputs[0].shape).to(device)
  m = nn.ReLU()
  for i in range(inputs[0].shape[0]):
    weights1 = inputs[0][i].to(device)
    weights2 = inputs[1][i].to(device)

    normed1 = weights1/weights1[weights1.argmax(-1)].to(device)
    normed2 = weights2/weights2[weights2.argmax(-1)].to(device)
    output[i] = normed1+normed2
  return output    


# ResNet 2 Electric Boogaloo

In [ ]:
label2id = {"Hateful":1, "Not Hateful" :0}
id2label = {1: "Hateful", 0: "Not Hateful"}

# ITS TRAINING TIME

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
modelResNet = ResNetForImageClassification.from_pretrained("microsoft/resnet-152", label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True).to(device)


image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-152")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

optimizerRes = AdamW(modelResNet.parameters(), lr=lr)
optimizerBert = AdamW(model.parameters(), lr=lr)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
modelResNet.to(device)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-152 and are newly initialized because the shapes did not match:
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([2, 2048]) in the model instantiated
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
criterion = nn.CrossEntropyLoss()
model.train()
modelResNet.train()

for e in range(epochs):

  # training
  train_losses = torch.zeros(len(train_h5_dataloader)).to(device)
  train_predictions = torch.Tensor().to(device)
  train_labels = torch.Tensor().to(device)

  for i, data in tqdm(enumerate(train_h5_dataloader), total=len(train_h5_dataloader), desc="Epoch " + str(e+1) + " Training"):

    text = data['text']
    image = data["image"].to(device)
    labels = data['label'].to(device).to(torch.int64).to(device)

    optimizerRes.zero_grad()
    optimizerBert.zero_grad()

    tokens = tokenizer(text, return_tensors='pt', padding=True).to(device)
    scoresBert = model(**tokens)

    with torch.no_grad():
        inputs = image_processor(image, return_tensors="pt").to(device)
    scoresRes = modelResNet(**inputs)

    
    lossRes = criterion(scoresRes.logits, labels.squeeze(0))
    lossRes.backward()
    optimizerRes.step()

    lossBert = criterion(scoresBert.logits, labels.squeeze(0))
    lossBert.backward()
    optimizerBert.step()

    combined_log = weighted((scoresRes.logits, scoresBert.logits))
    predictions = combined_log.argmax(dim=-1)

    train_losses[i] = lossRes + lossBert
    train_predictions = torch.cat((train_predictions, predictions))
    train_labels = torch.cat((train_labels, labels))

train_average_loss, train_accuracy, train_auroc = get_metrics_and_losses(train_losses, train_predictions, train_labels.squeeze(0))


Epoch 7 Training: 100%|██████████| 266/266 [06:02<00:00,  1.36s/it]


In [ ]:
train_average_loss, train_accuracy, train_auroc = get_metrics_and_losses(train_losses, train_predictions, train_labels.squeeze(0))
print(train_average_loss, train_accuracy, train_auroc)

0.25012505054473877 0.9790588235294118 tensor(0.9736, device='cuda:0', dtype=torch.float64)


# Lets Validation Some Stuff!


In [ ]:
validate_losses = torch.zeros(len(validate_dataloader)).to(device)
validate_predictions = torch.Tensor().to(device)
validate_labels = torch.Tensor().to(device)

for e in range(epochs):
  with torch.no_grad():
    for i, data in tqdm(enumerate(validate_dataloader), total=len(validate_dataloader), desc="Epoch " + str(e+1) + " Training"):

      # get inputs
      text = data['text']
      image = list(data['image'].detach().numpy())
      labels = data['label'].to(device).to(torch.int64)

      tokens = tokenizer(text, return_tensors='pt', padding=True).to(device)
      scoresBert = model(**tokens)

      with torch.no_grad():
          inputs = image_processor(image, return_tensors="pt").to(device)
      scoresRes = modelResNet(**inputs)

      combined_log = weighted((scoresRes.logits, scoresBert.logits))
      predictions = combined_log.argmax(dim=-1)

      # loss
      loss = criterion(combined_log, labels)

      # track values for metric logging
      validate_losses[i] = loss
      validate_predictions = torch.cat((validate_predictions, predictions), dim=0)
      validate_labels = torch.cat((validate_labels, labels), dim=0)


Epoch 7 Training: 100%|██████████| 16/16 [00:07<00:00,  2.19it/s]


In [ ]:
validate_average_loss, validate_accuracy, validate_auroc = get_metrics_and_losses(validate_losses, validate_predictions, validate_labels)
print(validate_average_loss, validate_accuracy, validate_auroc)

1.4141254425048828 0.5491428571428572 tensor(0.5453, device='cuda:0', dtype=torch.float64)


# Just Bert

In [ ]:
modelBertSolo = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


optimizerBertSolo = AdamW(modelBertSolo.parameters(), lr=lr)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
modelBertSolo.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
criterion = nn.CrossEntropyLoss()
modelBertSolo.train()
for e in range(epochs):

  # training
  train_losses = torch.zeros(len(train_h5_dataloader)).to(device)
  train_predictions = torch.Tensor().to(device)
  train_labels = torch.Tensor().to(device)

  for i, data in tqdm(enumerate(train_h5_dataloader), total=len(train_h5_dataloader), desc="Epoch " + str(e+1) + " Training"):

    text = data['text']
    image = data["image"].to(device)
    labels = data['label'].to(device).to(torch.int64).to(device)

   
    optimizerBertSolo.zero_grad()

    tokens = tokenizer(text, return_tensors='pt', padding=True).to(device)
    scoresBert = modelBertSolo(**tokens)
    

    lossBert = criterion(scoresBert.logits, labels.squeeze(0))
    lossBert.backward()
    optimizerBertSolo.step()

    predictions = scoresBert.logits.argmax(dim=-1)

    train_losses[i] = lossBert
    train_predictions = torch.cat((train_predictions, predictions))
    train_labels = torch.cat((train_labels, labels))

train_average_loss, train_accuracy, train_auroc = get_metrics_and_losses(train_losses, train_predictions, train_labels.squeeze(0))

Epoch 10 Training: 100%|██████████| 266/266 [01:22<00:00,  3.24it/s]


In [ ]:
train_average_loss, train_accuracy, train_auroc = get_metrics_and_losses(train_losses, train_predictions, train_labels.squeeze(0))
print(train_average_loss, train_accuracy, train_auroc)

0.19103385508060455 0.882 tensor(0.8761, device='cuda:0', dtype=torch.float64)


In [ ]:
validate_losses = torch.zeros(len(validate_dataloader)).to(device)
validate_predictions = torch.Tensor().to(device)
validate_labels = torch.Tensor().to(device)

for e in range(epochs):
  with torch.no_grad():
    for i, data in tqdm(enumerate(validate_dataloader), total=len(validate_dataloader), desc="Epoch " + str(e+1) + " Training"):

      # get inputs
      text = data['text']
      image = list(data['image'].detach().numpy())
      labels = data['label'].to(device).to(torch.int64)

      tokens = tokenizer(text, return_tensors='pt', padding=True).to(device)
      scoresBert = modelBertSolo(**tokens)
      
      predictions = scoresBert.logits.argmax(dim=-1)

      # loss
      loss = criterion(scoresBert.logits, labels)

      # track values for metric logging
      validate_losses[i] = loss
      validate_predictions = torch.cat((validate_predictions, predictions), dim=0)
      validate_labels = torch.cat((validate_labels, labels), dim=0)


Epoch 10 Training: 100%|██████████| 16/16 [00:01<00:00, 11.37it/s]


In [ ]:
validate_average_loss, validate_accuracy, validate_auroc = get_metrics_and_losses(validate_losses, validate_predictions, validate_labels)
print(validate_average_loss, validate_accuracy, validate_auroc)

1.6542060375213623 0.562 tensor(0.5586, device='cuda:0', dtype=torch.float64)


# Just Resnet


In [ ]:

modelResNetSolo = ResNetForImageClassification.from_pretrained("microsoft/resnet-152", label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True).to(device)


image_processor_solo = AutoImageProcessor.from_pretrained("microsoft/resnet-152")


optimizerResSolo = AdamW(modelResNetSolo.parameters(), lr=lr)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
modelResNetSolo.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
modelResNetSolo.train()

for e in range(epochs):

  # training
  train_losses = torch.zeros(len(train_h5_dataloader)).to(device)
  train_predictions = torch.Tensor().to(device)
  train_labels = torch.Tensor().to(device)

  for i, data in tqdm(enumerate(train_h5_dataloader), total=len(train_h5_dataloader), desc="Epoch " + str(e+1) + " Training"):

    text = data['text']
    image = data["image"].to(device)
    labels = data['label'].to(device).to(torch.int64).to(device)

    optimizerResSolo.zero_grad()

    with torch.no_grad():
        inputs = image_processor_solo(image, return_tensors="pt").to(device)
    scoresRes = modelResNetSolo(**inputs)

    
    lossRes = criterion(scoresRes.logits, labels.squeeze(0))
    lossRes.backward()
    optimizerResSolo.step()


    predictions = scoresRes.logits.argmax(dim=-1)

    train_losses[i] = lossRes
    train_predictions = torch.cat((train_predictions, predictions))
    train_labels = torch.cat((train_labels, labels))

train_average_loss, train_accuracy, train_auroc = get_metrics_and_losses(train_losses, train_predictions, train_labels.squeeze(0))

Epoch 10 Training: 100%|██████████| 266/266 [03:53<00:00,  1.14it/s]


In [ ]:
train_average_loss, train_accuracy, train_auroc = get_metrics_and_losses(train_losses, train_predictions, train_labels.squeeze(0))
print(train_average_loss, train_accuracy, train_auroc)

0.14727601408958435 0.9369411764705883 tensor(0.9312, device='cuda:0', dtype=torch.float64)


In [ ]:
validate_losses = torch.zeros(len(validate_dataloader)).to(device)
validate_predictions = torch.Tensor().to(device)
validate_labels = torch.Tensor().to(device)

for e in range(epochs):
  with torch.no_grad():
    for i, data in tqdm(enumerate(validate_dataloader), total=len(validate_dataloader), desc="Epoch " + str(e+1) + " Training"):

      # get inputs
      text = data['text']
      image = list(data['image'].detach().numpy())
      labels = data['label'].to(device).to(torch.int64)

      with torch.no_grad():
          inputs = image_processor_solo(image, return_tensors="pt").to(device)
      scoresRes = modelResNetSolo(**inputs)

      predictions = scoresRes.logits.argmax(dim=-1)

      # loss
      loss = criterion(scoresRes.logits, labels)

      # track values for metric logging
      validate_losses[i] = loss
      validate_predictions = torch.cat((validate_predictions, predictions), dim=0)
      validate_labels = torch.cat((validate_labels, labels), dim=0)


Epoch 10 Training: 100%|██████████| 16/16 [00:05<00:00,  2.89it/s]


In [ ]:
validate_average_loss, validate_accuracy, validate_auroc = get_metrics_and_losses(validate_losses, validate_predictions, validate_labels)
print(validate_average_loss, validate_accuracy, validate_auroc)

1.8739736080169678 0.5124 tensor(0.5098, device='cuda:0', dtype=torch.float64)
